In [2]:
import random
from datetime import datetime, timedelta
from typing import List, Tuple, Dict
import numpy as np
from datasets import Dataset, Features, Value, Sequence
import re
from typing import List
from functools import cmp_to_key
import os

from datasets import load_from_disk
%pdb on


Automatic pdb calling has been turned ON


In [3]:
from datasets import load_from_disk
from transformer_reasoning.utils import get_project_root
import random

def load_and_inspect_datasets():
    # Load the bios dataset
    bios_dataset = load_from_disk(str(get_project_root() / "generated_data/bios/bios_dataset_10000"))

    # Load the QA dataset
    qa_dataset = load_from_disk(str(get_project_root() / "generated_data/qa_dataset_10000"))

    # Inspect bios dataset
    print("Bios Dataset:")
    print(f"Number of entries: {len(bios_dataset)}")
    print("\nSample entries from bios dataset:")
    for i in random.sample(range(len(bios_dataset)), 3):
        print(f"\nEntry {i}:")
        print(f"Name: {bios_dataset[i]['name']}")
        print(f"Bio: {bios_dataset[i]['bio']}")
        print(f"Birth Date: {bios_dataset[i]['birth_date']}")
        print(f"Birth City: {bios_dataset[i]['birth_city']}")

    # Inspect QA dataset
    print("\n\nQA Dataset:")
    for split in qa_dataset.keys():
        if len(qa_dataset[split]) > 0:
            print(f"\n{split.capitalize()} split:")
            print(f"Number of entries: {len(qa_dataset[split])}")
            print("Sample entries:")
            for i in random.sample(range(len(qa_dataset[split])), 3):
                print(f"\nEntry {i}:")
                print(f"Question: {qa_dataset[split][i]['questions.question']}")
                print(f"Answer: {qa_dataset[split][i]['questions.answer']}")
                print(f"Order: {qa_dataset[split][i]['questions.order']}")

if __name__ == "__main__":
    load_and_inspect_datasets()

Bios Dataset:
Number of entries: 10000000

Sample entries from bios dataset:

Entry 4726568:
Name: Angie Emelia Bender
Bio: Angie Emelia Bender is a researcher who earned their degree from Nanjing University. Katrice Valentina Appleberry is the subject's worst enemy. Terrence Latosha Mc is the daughter of the subject. Semen Indonesia is where they're currently grinding. Roy Alphonse Degrand is the subject's primary social associate and confidant. Brisbane, Queensland—that's where they popped out. 1949-04-01 saw the arrival of this bigwig.
Birth Date: 1949-04-01 00:00:00
Birth City: Brisbane, Queensland

Entry 2405275:
Name: Sun Trudi Beagan
Bio: At Beijing University of Chemical Technology, Sun Trudi Beagan was known for their intellectual prowess. Yessenia Alejandra Paronto stands as the formidable nemesis to the subject. Ellan Jane Toothman is their kid, a real handful but full of life. Semen Indonesia is where the subject currently excels. Meet Alla Marisol Burriss, the wingman who 

In [1]:
from datasets import load_from_disk
from transformer_reasoning.utils import get_project_root
import glob
import os

# Get all profile datasets
dataset_paths = glob.glob(str(get_project_root() / "generated_data/profiles_dataset_*"))

for path in dataset_paths:
    # Load dataset
    dataset = load_from_disk(path)
    
    # Get N from filename
    N = os.path.basename(path).split('_')[-1]
    
    # Push to hub
    dataset.push_to_hub(f"EleutherAI/profiles_dataset_{N}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/250 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]